In [1]:
# Initialize duckdb
import duckdb
import polars as pl

parquet_file = "train.parquet"

# Start a DuckDB session
con = duckdb.connect(database="my_duckdb.db", read_only=False)

# Create a virtual table that points to the parquet file
con.execute(
    f"CREATE VIEW IF NOT EXISTS train AS SELECT * FROM parquet_scan('{parquet_file}')"
)

df = con.execute(
    """SELECT molecule_smiles as smiles, protein_name, binds FROM train"""
).fetch_arrow_table()

df = pl.from_arrow(df)

In [2]:
BRD4_records = df.filter(df['protein_name'] == 'BRD4')
HSA_records = df.filter(df['protein_name'] == 'HSA')
sEH_records = df.filter(df['protein_name'] == 'sEH')

print(BRD4_records.shape)
print(HSA_records.shape)
print(sEH_records.shape)


(98415610, 3)
(98415610, 3)
(98415610, 3)


In [3]:
BRD4_binded_records_count = BRD4_records.filter(BRD4_records['binds'] == 1).shape[0]
HSA_binded_records_count = HSA_records.filter(HSA_records['binds'] == 1).shape[0]
sEH_binded_records_count = sEH_records.filter(sEH_records['binds'] == 1).shape[0]

print(f"Number of BRD4 records that bind: {BRD4_binded_records_count}")
print(f"Number of HSA records that bind: {HSA_binded_records_count}")
print(f"Number of sEH records that bind: {sEH_binded_records_count}")

Number of BRD4 records that bind: 456964
Number of HSA records that bind: 408410
Number of sEH records that bind: 724532


In [4]:
frac_for_positives = 0.1  # 10% for positives
frac_for_negatives = 0.01  # 1% for negatives

# Filter the DataFrame into two based on 'Binds'
positives = BRD4_records.filter(pl.col("binds") == 1)
negatives = BRD4_records.filter(pl.col("binds") == 0)

print(positives.head())
print(negatives.head())

shape: (5, 3)
┌─────────────────────────────────┬──────────────┬───────┐
│ smiles                          ┆ protein_name ┆ binds │
│ ---                             ┆ ---          ┆ ---   │
│ str                             ┆ str          ┆ i64   │
╞═════════════════════════════════╪══════════════╪═══════╡
│ C#CC[C@@H](CC(=O)N[Dy])Nc1nc(N… ┆ BRD4         ┆ 1     │
│ C#CC[C@@H](CC(=O)N[Dy])Nc1nc(N… ┆ BRD4         ┆ 1     │
│ C#CC[C@@H](CC(=O)N[Dy])Nc1nc(N… ┆ BRD4         ┆ 1     │
│ C#CC[C@@H](CC(=O)N[Dy])Nc1nc(N… ┆ BRD4         ┆ 1     │
│ C#CC[C@@H](CC(=O)N[Dy])Nc1nc(N… ┆ BRD4         ┆ 1     │
└─────────────────────────────────┴──────────────┴───────┘
shape: (5, 3)
┌─────────────────────────────────┬──────────────┬───────┐
│ smiles                          ┆ protein_name ┆ binds │
│ ---                             ┆ ---          ┆ ---   │
│ str                             ┆ str          ┆ i64   │
╞═════════════════════════════════╪══════════════╪═══════╡
│ C#CCOc1ccc(CNc2nc(NCC3CCCN

In [5]:
import polars as pl

# Define the fractions for positives and negatives
frac_for_positives = 0.1  # 10% for positive
frac_for_negatives = 0.01  # 1% for negatives

# Sample the positives dataframe
positives_sampled = positives.sample(
    fraction=frac_for_positives,
    shuffle=True,
)

# Sample the negatives dataframe
negatives_sampled = negatives.sample(
    fraction=frac_for_negatives,
    shuffle=True,
)

# Concatenate the sampled dataframes
sampled_df = pl.concat([positives_sampled, negatives_sampled])
# Reset the index of the sampled dataframe sampled_df = sampled_df.reset_index(drop=True) # Print the sampled dataframe print(sampled_df)

print(sampled_df.head())

shape: (5, 3)
┌─────────────────────────────────┬──────────────┬───────┐
│ smiles                          ┆ protein_name ┆ binds │
│ ---                             ┆ ---          ┆ ---   │
│ str                             ┆ str          ┆ i64   │
╞═════════════════════════════════╪══════════════╪═══════╡
│ COc1ncc(Nc2nc(Nc3nnn[nH]3)nc(N… ┆ BRD4         ┆ 1     │
│ Cc1cccc(CCCNc2nc(Nc3ccc(=O)n(C… ┆ BRD4         ┆ 1     │
│ CC(CNc1nc(Nc2cc(Cl)ncn2)nc(NC(… ┆ BRD4         ┆ 1     │
│ CCCCC(Nc1nc(Nc2ccc(C(C)(C)C#N)… ┆ BRD4         ┆ 1     │
│ COCc1ccccc1CNc1nc(NCC(O)CS(C)=… ┆ BRD4         ┆ 1     │
└─────────────────────────────────┴──────────────┴───────┘


In [7]:
import polars as pl
import xgboost as xgb
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score

# Function to convert SMILES to bit vector
def smiles_to_bitvec(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        bit_vec = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=1024)
        return np.array(list(map(int, bit_vec.ToBitString())))  # Convert to NumPy array of integers
    return np.zeros(1024, dtype=int)  # Return an array of zeros if SMILES is invalid


# Apply the function and prepare the dataset
df = sampled_df.with_columns(
    pl.col("smiles").map_elements(smiles_to_bitvec).alias("ecfp")
)
# Extract features and labels
X = np.stack(df["ecfp"].to_numpy())
y = df["binds"].to_numpy()

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define XGBoost parameters
params = {
    'device': 'cuda',
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'learning_rate': 0.1,
    'max_depth': 5,
    'alpha': 10,
    'n_estimators': 100
}

# Train the model
dtrain = xgb.DMatrix(X_train, label=Y_train)
model = xgb.train(params, dtrain, num_boost_round=10)

# Save the model
model.save_model("model.bin")

# Validate the model
dtest = xgb.DMatrix(X_test, label=Y_test)
y_pred = model.predict(dtest)

y_pred_binary = (y_pred > 0.5).astype(int)
average_precision = average_precision_score(Y_test, y_pred_binary)
print("Average precision:", average_precision)


C:\Users\Zach PC\AppData\Local\Temp\ipykernel_31812\3070957975.py:19: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df = sampled_df.with_columns(
c:\Users\Zach PC\kaggle_projects\BELKA_comp\.venv\Lib\site-packages\xgboost\core.py:160: UserWarning: [14:09:03] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
c:\Users\Zach PC\kaggle_projects\BELKA_comp\.venv\Lib\site-packages\xgboost\core.py:160: UserWarning: [14:09:03] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Pa

Average precision: 0.1884010336330857


In [ ]:
#Score to beat: 0.1884